In [18]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import sympy as sp
import handcalcs.render
import forallpeople as si
import matplotlib.pyplot as plt
%matplotlib inline
from math import tan, sin,cos, pi, radians, sqrt, trunc
from IPython.display import display, Markdown
si.environment('structural', top_level=True)

<h2>KBE Beam verification</h2>

<h3>Compartment: Gym</h3>

* Add picture of beam

<h4>Loads</h4>

In [19]:
%%render 2

psi_11 = 0.4
G_k = 27.49/1.1*kN/m
Q_k = 5.8*4.6*kN/m

<IPython.core.display.Latex object>

Load combination for fire:

In [20]:
%%render 2
q_d = G_k+psi_11*Q_k

<IPython.core.display.Latex object>

<h4>Beam parameters</h4>

In [33]:
%%render param 2

L = 9.6*m
b = 270*mm
d = 585*mm
A_t = 2.34*10**5*mm**2
fck_20 = 45*MPa
fyk_20 = 500*MPa
phi_s = 12.5*mm
A_w = 93*mm**2
A_p = 16*A_w
E_s20 = 195*GPa


<IPython.core.display.Latex object>

From CONFIRE

* Add correct cross section geometry in CONFIRE and compartment properties

In [22]:
%%render param 2

T_max = (284*dC)
T_cold = (286*dC) 
xi_CMH = 0.9983
xi_CMC = 0.9659

<IPython.core.display.Latex object>

In [23]:
cte = pd.read_excel('strength-reduction-constants.xlsx')

In [24]:
cte

,Material,k,T1,T2,T8,T64
0,Hot rolled bars 0.2 % stress,0.000,6000,620,565,1100
1,Hot rolled bars 2.0 % stress,0.000,100000,100000,593,100000
2,Hot rolled bars 0.2 % residual stress,1.000,100000,100000,100000,100000
3,Hot rolled bars 2.0 % residual stress,1.000,100000,100000,100000,100000
4,Cold worked bars 0.2 % stress,0.000,100000,900,555,100000
5,Cold worked bars 2.0 % stress,0.000,100000,5000,560,100000
6,Cold worked bars 0.2 % residual stress,0.580,100000,5000,590,730
7,Cold worked bars 2.0 % residual stress,0.520,100000,1500,580,650
8,C-w prestressing steel 0.2 % stress,0.000,2000,360,430,100000
9,C-w prestressing steel 2.0 % stress,0.000,100000,490,450,100000


In [25]:
def material_constants(mat_index):
    k = cte['k '][mat_index]
    T1 = cte['T1 '][mat_index]*dC
    T2 = cte['T2 '][mat_index]*dC
    T8 = cte['T8 '][mat_index]*dC
    T64 = cte['T64'][mat_index]*dC
    return k, T1, T2, T8, T64



In [26]:
from handcalcs import handcalc
@handcalc(jupyter_display = True)
def material_deterioration(T,k,T1,T2,T8,T64):
    """
    Calculate the deterioration of a material.
    """
    xi = k + (1-k)/(1+T/T1+(T/T2)**2+(T/T8)**8+(T/T64)**64)
    return xi


In [27]:
k, T1, T2, T8, T64 = material_constants(8)
xi_sH = material_deterioration(T_max,k,T1,T2,T8,T64)
k, T1, T2, T8, T64 = material_constants(10)
xi_sC = material_deterioration(T_cold,k,T1,T2,T8,T64)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

**Before fire**

Using effective prestress strength per wire = 110 kN, provided by the supplier

In [41]:
%%render 2

M_max_fi = q_d*L**2/8
F_su20 = (110*kN)
y_20 = 16*F_su20/(fck_20*b)
M_u20 = 16*F_su20*(d-y_20/2)
mu_20 = M_max_fi/M_u20

<IPython.core.display.Latex object>

**HOT - During fire**

In [29]:
%%render 2

F_sufi = xi_sH*F_su20
y_H = F_sufi/(xi_CMH*fck_20*b)
M_u_fi = F_sufi*(d-0.5*y_H)
mu_fi = M_max_fi/M_u_fi

<IPython.core.display.Latex object>

**COLD - After fire**

In [30]:
%%render 2

F_suC = xi_sC*F_su20
y_C = F_suC/(xi_CMC*fck_20*b)
M_u_C = F_suC*(d-y_C/2)
mu_C = M_max_fi/M_u_C

<IPython.core.display.Latex object>

Over-reinforcment check

In [31]:
%%render 2

varepsilon_s = (d-5/4*y_C)/(5/4*y_C)*0.35/100/xi_CMC
varepsilon_smin = fyk_20/E_s20

<IPython.core.display.Latex object>

In [32]:
if varepsilon_s>varepsilon_smin and mu_fi<1:
    display(Markdown('The beam is verified for fire'))
else:
    display(Markdown('The beam is not verified for fire'))

The beam is not verified for fire